In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/home/bhavya/Downloads/datasets/pcos_dataset.csv')
df.head()

,Age,BMI,Menstrual_Irregularity,Testosterone_Level(ng/dL),Antral_Follicle_Count,PCOS_Diagnosis
0,24,34.7,1,25.2,20,0
1,37,26.4,0,57.1,25,0
2,32,23.6,0,92.7,28,0
3,28,28.8,0,63.1,26,0
4,25,22.1,1,59.8,8,0


In [3]:
df.shape

(1000, 6)

In [22]:
# df.drop_duplicates(inplace=True) -> no duplicates found
# df.shape
# pd.isnull(df).sum() -> no null values found

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Age                        1000 non-null   int64  
 1   BMI                        1000 non-null   float64
 2   Menstrual_Irregularity     1000 non-null   int64  
 3   Testosterone_Level(ng/dL)  1000 non-null   float64
 4   Antral_Follicle_Count      1000 non-null   int64  
 5   PCOS_Diagnosis             1000 non-null   int64  
dtypes: float64(2), int64(4)
memory usage: 47.0 KB


In [15]:
df['patient_id'] = range(1, len(df) + 1)  # adding a patient id column
df.set_index('patient_id', inplace=True)  # Setting it as the index

In [16]:
df.sample(5)

,Age,BMI,Menstrual_Irregularity,Testosterone_Level(ng/dL),Antral_Follicle_Count,PCOS_Diagnosis
patient_id,,,,,,
565,22,32.9,1,48.8,13,1
448,44,23.4,1,20.7,9,0
813,45,26.8,1,72.4,11,1
198,30,30.2,0,70.7,27,0
423,34,30.4,0,96.5,10,0


In [24]:
df = df.rename(columns = {'Testosterone_Level(ng/dL)' : 'Testosterone_Level'})

In [26]:
# converting cleaned dataset into another CSV file for SQL analysis in BigQuery
df.to_csv('cleaned_pcos.csv', index=True)